In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [3]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [7]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='df54ad96-c516-409e-bc86-d324036a2bad'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "My favourite colour is green". We need to update the favourite colour in the state using the provided function update_favourite_colour. So we call that function with favourite_colour: "green".', 'tool_calls': [{'id': 'fc_2a58a705-d726-47dc-89a0-31980fca780f', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 137, 'total_tokens': 210, 'completion_time': 0.151593911, 'completion_tokens_details': {'reasoning_tokens': 42}, 'prompt_time': 0.005371741, 'prompt_tokens_details': None, 'queue_time': 0.004449109, 'total_time': 0.156965652}, 'model_name': 'openai/gpt-oss-120b', 'syst

In [8]:
print(response["messages"][-1].content)

Got it—your favourite colour is now set to **green**! If there’s anything else you’d like to share or ask, just let me know.


In [9]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='8c7229dc-3148-41f6-afe9-675a35632e1d'),
              AIMessage(content="Hello! I'm doing great, thank you for asking. How can I help you today?", additional_kwargs={'reasoning_content': 'The user greets. We respond friendly. No need to use function.'}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 138, 'total_tokens': 181, 'completion_time': 0.088698068, 'completion_tokens_details': {'reasoning_tokens': 16}, 'prompt_time': 0.005192656, 'prompt_tokens_details': None, 'queue_time': 0.004703648, 'total_time': 0.093890724}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_49bfac06f1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c5301-a8dd-7692-a2aa-045cd7732d8d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 138, 'ou

In [10]:
print(response["messages"][-1].content)

Hello! I'm doing great, thank you for asking. How can I help you today?


## Read state

In [11]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    model=model,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [12]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='9e2be81a-93c9-4051-9591-77eec4361eb3'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "My favourite colour is green". We need to store this in state using update_favourite_colour. Then maybe respond acknowledging. Use function.', 'tool_calls': [{'id': 'fc_275ae760-111e-4f24-ad03-39a652c90ba9', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 161, 'total_tokens': 224, 'completion_time': 0.129830659, 'completion_tokens_details': {'reasoning_tokens': 32}, 'prompt_time': 0.006046436, 'prompt_tokens_details': None, 'queue_time': 0.004538143, 'total_time': 0.135877095}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_178e424650', 'service_tier': 'o

In [13]:
print(response["messages"][-1].content)

Got it! I’ve noted that your favourite colour is **green**. 🌿 If there’s anything else you’d like to share or ask, just let me know!


In [14]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='9e2be81a-93c9-4051-9591-77eec4361eb3'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "My favourite colour is green". We need to store this in state using update_favourite_colour. Then maybe respond acknowledging. Use function.', 'tool_calls': [{'id': 'fc_275ae760-111e-4f24-ad03-39a652c90ba9', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 161, 'total_tokens': 224, 'completion_time': 0.129830659, 'completion_tokens_details': {'reasoning_tokens': 32}, 'prompt_time': 0.006046436, 'prompt_tokens_details': None, 'queue_time': 0.004538143, 'total_time': 0.135877095}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_178e424650', 'service_tier': 'o

In [15]:
print(response["messages"][-1].content)

Your favourite colour is **green**. 🌱
